# Appendix

In [2]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import duckdb

In [3]:
#import packages and read csv file
raw_df=pd.read_csv("product_info.csv")

print(raw_df.shape)
raw_df.head()

(8494, 27)


,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
0,P473671,Fragrance Discovery Set,6342,19-69,6320,3.6364,11.0,NaN,NaN,NaN,...,1,0,0,"['Unisex/ Genderless Scent', 'Warm &Spicy Scen...",Fragrance,Value & Gift Sets,Perfume Gift Sets,0,NaN,NaN
1,P473668,La Habana Eau de Parfum,6342,19-69,3827,4.1538,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,85.0,30.0
2,P473662,Rainbow Bar Eau de Parfum,6342,19-69,3253,4.2500,16.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
3,P473660,Kasbah Eau de Parfum,6342,19-69,3018,4.4762,21.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
4,P473658,Purple Haze Eau de Parfum,6342,19-69,2691,3.2308,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0


## Part 3: Data Cleaning: <a name="cleaning"></a>
Based on the research questions and my interests, I filtered out 7 variables/columns relevant to my study, including product name, brand name, loves count, rating, price, primary category, and secondary category.

- Null Values: I examined the dataset for null values and subsequently dropped 278 values. Given the size of this dataframe of over 8,000 values, dropping a small fraction would not be relevant enough to make a difference.
- Standardizing: I changed ratings and price_usd to two decimal values to ensure standardization in the real world
- Feature Engineering (Brand Names): I categorized brand names with fewer than 10 associated products as 'Other.' I want to optimize the price prediction model as singular values for each brand proved less informative for predictive modeling.
- Outliers: I removed outliers in love_counts column based on its z-scores. This decision was motivated by the considerable variability observed in the dataset.
- Feature Engineering (Primary Category): To enhance the relevance of the 'primary_category' column, non-categorical variables such as 'Mini Size' and 'Men' were reassigned to more meaningful values derived from the 'secondary_category.' Concurrently, categories with a limited product representation were consolidated into an 'Other' category.
- Duplicates: I checked and removed duplicate rows for the clean data preprocessing.

In [4]:
#Drop features that are not relevant to my research questions

dropped_df= raw_df.drop(['product_id','brand_id','reviews','size', 'variation_type', 'variation_value', \
                         'variation_desc', 'value_price_usd', 'sale_price_usd', 'limited_edition', 'new',\
                         'online_only','out_of_stock', 'sephora_exclusive', 'child_count','child_max_price', \
                         'child_min_price', 'highlights', 'tertiary_category', 'ingredients'],axis='columns')
dropped_df.shape
dropped_df.head()


,product_name,brand_name,loves_count,rating,price_usd,primary_category,secondary_category
0,Fragrance Discovery Set,19-69,6320,3.6364,35.0,Fragrance,Value & Gift Sets
1,La Habana Eau de Parfum,19-69,3827,4.1538,195.0,Fragrance,Women
2,Rainbow Bar Eau de Parfum,19-69,3253,4.2500,195.0,Fragrance,Women
3,Kasbah Eau de Parfum,19-69,3018,4.4762,195.0,Fragrance,Women
4,Purple Haze Eau de Parfum,19-69,2691,3.2308,195.0,Fragrance,Women


### Missing Values

In [5]:
#Check number of null values
dropped_df.isnull().sum()

product_name            0
brand_name              0
loves_count             0
rating                278
price_usd               0
primary_category        0
secondary_category      8
dtype: int64

In [6]:
#Create new dataframe after dropping null values. 
#Since our data frame has over 8,000 values, dropping 278 values will not make a difference in our data analysis.
new_df = dropped_df.dropna()

#Double check if all null values are dropped
new_df.isnull().sum()

product_name          0
brand_name            0
loves_count           0
rating                0
price_usd             0
primary_category      0
secondary_category    0
dtype: int64

In [7]:
#Check shape of our new data frame
print(new_df.shape)
new_df.head()

(8209, 7)


,product_name,brand_name,loves_count,rating,price_usd,primary_category,secondary_category
0,Fragrance Discovery Set,19-69,6320,3.6364,35.0,Fragrance,Value & Gift Sets
1,La Habana Eau de Parfum,19-69,3827,4.1538,195.0,Fragrance,Women
2,Rainbow Bar Eau de Parfum,19-69,3253,4.2500,195.0,Fragrance,Women
3,Kasbah Eau de Parfum,19-69,3018,4.4762,195.0,Fragrance,Women
4,Purple Haze Eau de Parfum,19-69,2691,3.2308,195.0,Fragrance,Women


### Standardizing

In [8]:
#Change price and rating to two decimal places

new_df['price_usd'].unique()
new_df.head()

pd.set_option('display.float_format', '{:.2f}'.format)
new_df.head()


,product_name,brand_name,loves_count,rating,price_usd,primary_category,secondary_category
0,Fragrance Discovery Set,19-69,6320,3.64,35.00,Fragrance,Value & Gift Sets
1,La Habana Eau de Parfum,19-69,3827,4.15,195.00,Fragrance,Women
2,Rainbow Bar Eau de Parfum,19-69,3253,4.25,195.00,Fragrance,Women
3,Kasbah Eau de Parfum,19-69,3018,4.48,195.00,Fragrance,Women
4,Purple Haze Eau de Parfum,19-69,2691,3.23,195.00,Fragrance,Women


### Feature Engineering with Brand Name

In [9]:
#Check for the count of each unique brand name.

print(len(new_df['brand_name'].unique()))
new_df.loc[:, 'brand_name'] = new_df['brand_name'].apply(lambda x: x.strip())
brand_stats = new_df['brand_name'].value_counts(ascending=False)
brand_stats

302


brand_name
SEPHORA COLLECTION    346
CLINIQUE              177
Dior                  133
tarte                 126
Bumble and bumble     107
                     ... 
Aquis                   1
dpHUE                   1
Eight & Bob             1
PROVEN Skincare         1
Jillian Dempsey         1
Name: count, Length: 302, dtype: int64

In [10]:
#Change brand names with less than 10 products to Other.

brand_stats_less_than_10 = brand_stats[brand_stats <= 10]

new_df.loc[:, 'brand_name'] = new_df['brand_name'].apply(lambda x: 'Other' if x in brand_stats_less_than_10 else x)

#Double check this works
updated_brand_stats = new_df['brand_name'].value_counts(ascending=False)
print(len(updated_brand_stats[updated_brand_stats <= 10]))

new_df.head()

0


,product_name,brand_name,loves_count,rating,price_usd,primary_category,secondary_category
0,Fragrance Discovery Set,19-69,6320,3.64,35.00,Fragrance,Value & Gift Sets
1,La Habana Eau de Parfum,19-69,3827,4.15,195.00,Fragrance,Women
2,Rainbow Bar Eau de Parfum,19-69,3253,4.25,195.00,Fragrance,Women
3,Kasbah Eau de Parfum,19-69,3018,4.48,195.00,Fragrance,Women
4,Purple Haze Eau de Parfum,19-69,2691,3.23,195.00,Fragrance,Women


### Outliers

In [11]:
# Check variability of love_counts

new_df.loves_count.describe()

count      8209.00
mean      30160.74
std       67014.73
min           0.00
25%        4162.00
50%       10456.00
75%       27800.00
max     1401068.00
Name: loves_count, dtype: float64

In [12]:
# Find outliers with z-scores 
z_scores = (new_df['loves_count'] - new_df['loves_count'].mean()) / new_df['loves_count'].std()

# Filter out rows where the absolute z-score is greater than the threshold in a new dataframe
new_df_no_outliers = new_df[np.abs(z_scores) <= 3.0]

print(len(new_df_no_outliers))


8045


In [13]:
# Check variability after cleaning
new_df_no_outliers.loves_count.describe()

count     8045.00
mean     22800.56
std      33435.44
min          0.00
25%       4079.00
50%      10123.00
75%      26153.00
max     231052.00
Name: loves_count, dtype: float64

In [14]:
new_df_no_outliers.head()

,product_name,brand_name,loves_count,rating,price_usd,primary_category,secondary_category
0,Fragrance Discovery Set,19-69,6320,3.64,35.00,Fragrance,Value & Gift Sets
1,La Habana Eau de Parfum,19-69,3827,4.15,195.00,Fragrance,Women
2,Rainbow Bar Eau de Parfum,19-69,3253,4.25,195.00,Fragrance,Women
3,Kasbah Eau de Parfum,19-69,3018,4.48,195.00,Fragrance,Women
4,Purple Haze Eau de Parfum,19-69,2691,3.23,195.00,Fragrance,Women


### Feature Engineering with Category Name

In [15]:
# Check for Category Names
new_df_no_outliers['primary_category'].unique()

array(['Fragrance', 'Bath & Body', 'Mini Size', 'Hair', 'Makeup',
       'Skincare', 'Tools & Brushes', 'Men'], dtype=object)

In [16]:
# Look for total products in each category
category_counts = new_df_no_outliers['primary_category'].value_counts()
print(category_counts)

primary_category
Skincare           2316
Makeup             2215
Hair               1410
Fragrance          1334
Bath & Body         393
Mini Size           269
Men                  58
Tools & Brushes      50
Name: count, dtype: int64


In [17]:
# Replace Mini Size with the more detailed category in the Secondary Category so it's not misleading
new_values = new_df_no_outliers['primary_category'] == 'Mini Size'
new_df_no_outliers.loc[new_values, 'primary_category'] = new_df_no_outliers.loc[new_values, 'secondary_category']

# Look for total products in each category again
category_counts = new_df_no_outliers['primary_category'].value_counts()
print(category_counts)

primary_category
Skincare             2378
Makeup               2347
Hair                 1459
Fragrance            1348
Bath & Body           400
Men                    58
Tools & Brushes        50
Value & Gift Sets       5
Name: count, dtype: int64


In [18]:
# Replace Men with the more detailed category in the Secondary Category so it's not misleading

new_values_men = new_df_no_outliers['primary_category'] == 'Men'
new_df_no_outliers.loc[new_values_men, 'primary_category'] = new_df_no_outliers.loc[new_values_men, 'secondary_category']

# Double check for total products in each category again

category_counts = new_df_no_outliers['primary_category'].value_counts()
print(category_counts)

primary_category
Skincare             2410
Makeup               2347
Hair                 1462
Fragrance            1348
Bath & Body           400
Tools & Brushes        50
Shaving                14
Value & Gift Sets       9
Other Needs             5
Name: count, dtype: int64


In [19]:
# Combined Shaving and Tools & Brushes into own category since both are related

new_df_no_outliers['primary_category'] = new_df_no_outliers['primary_category'].replace({'Shaving': 'Tools & Brushes'})

category_counts = new_df_no_outliers['primary_category'].value_counts()
print(category_counts)

primary_category
Skincare             2410
Makeup               2347
Hair                 1462
Fragrance            1348
Bath & Body           400
Tools & Brushes        64
Value & Gift Sets       9
Other Needs             5
Name: count, dtype: int64


C:\Users\ashle\AppData\Local\Temp\ipykernel_22532\827364298.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_no_outliers['primary_category'] = new_df_no_outliers['primary_category'].replace({'Shaving': 'Tools & Brushes'})


In [20]:
# Replace 'Value & Gift Sets' and 'Other Needs' with 'Other' in the primary_category column
new_df_no_outliers['primary_category'] = new_df_no_outliers['primary_category'].replace({'Value & Gift Sets': 'Other', 'Other Needs': 'Other'})

category_counts = new_df_no_outliers['primary_category'].value_counts()
print(category_counts)

primary_category
Skincare           2410
Makeup             2347
Hair               1462
Fragrance          1348
Bath & Body         400
Tools & Brushes      64
Other                14
Name: count, dtype: int64


C:\Users\ashle\AppData\Local\Temp\ipykernel_22532\2207544356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_no_outliers['primary_category'] = new_df_no_outliers['primary_category'].replace({'Value & Gift Sets': 'Other', 'Other Needs': 'Other'})


In [21]:
# Drop secondary_category column after cleaning primary_category

final_df = new_df_no_outliers.drop(columns=['secondary_category'])

### Duplicates

In [22]:
# Display the rows that are duplicates

duplicates = final_df.duplicated()

# Display the rows that are duplicates
duplicate_rows = final_df[duplicates]
print(duplicate_rows)

Empty DataFrame
Columns: [product_name, brand_name, loves_count, rating, price_usd, primary_category]
Index: []


In [23]:
# Final dataframe
final_df.head()

,product_name,brand_name,loves_count,rating,price_usd,primary_category
0,Fragrance Discovery Set,19-69,6320,3.64,35.00,Fragrance
1,La Habana Eau de Parfum,19-69,3827,4.15,195.00,Fragrance
2,Rainbow Bar Eau de Parfum,19-69,3253,4.25,195.00,Fragrance
3,Kasbah Eau de Parfum,19-69,3018,4.48,195.00,Fragrance
4,Purple Haze Eau de Parfum,19-69,2691,3.23,195.00,Fragrance


In [24]:
final_df.to_csv('appendix.csv')